## **How to decide where to open your new shop with Python**

*Tania Cajal and Vladislav Kaleev, 2021*

For this project, we are taking on a little role play in which the owner of an Italian fashion store wants to open a boutique in Barcelona. Our client has asked us to find the optimum location for a clothing shop in the city. Basing ourselves on the discipline of New Economic Geography, we are going to locate existing fashion shops, available properties, and city demographics. 

In [2]:
# Importing libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import geojson
import h3
import folium
import osmnx as ox
from shapely import wkt
from folium.plugins import HeatMap
from shapely.geometry import Polygon
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import requests

In [3]:
# Getting the base map with Folium
Barcelona = folium.Map(location = [41.3887900, 2.1589900], zoom_start = 13)
Barcelona

### Part 1: Market Size
An important part of choosing a location for a store is evaluating the market size around it. How many people would have the store within walking distance? To answer this question, we will be using data from Barcelona's open data portal (https://opendata-ajuntament.barcelona.cat/data/es/dataset/taula-map-scensal).

In [4]:
# Reading the dataset
persones = pd.read_csv('2021_padro_ocupacio_mitjana.csv')

In [5]:
# Viewing and evaluating the data
persones

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Poblacio,Domicilis,Ocupacio_mitjana_(persones_ per_domicili)
0,2021,1,Ciutat Vella,1,el Raval,48688,17298,2.81
1,2021,1,Ciutat Vella,2,el Barri Gòtic,22850,6571,3.48
2,2021,1,Ciutat Vella,3,la Barceloneta,15125,6927,2.18
3,2021,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",23009,10071,2.28
4,2021,2,Eixample,5,el Fort Pienc,33445,12782,2.62
...,...,...,...,...,...,...,...,...
68,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,13536,5356,2.53
69,2021,10,Sant Martí,70,el Besòs i el Maresme,26116,8432,3.10
70,2021,10,Sant Martí,71,Provençals del Poblenou,21185,8315,2.55
71,2021,10,Sant Martí,72,Sant Martí de Provençals,26229,10584,2.48


In [6]:

#Creating a dataframe with address of locations we want to reterive 
df = pd.DataFrame({'add': persones['Nom_Barri']}) #Example Address shop

#Creating an instance of Nominatim Class
geolocator = Nominatim(user_agent="my_request")

#Applying the rate limiter wrapper
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

#Applying the method to pandas DataFrame
df['location'] = df['add'].apply(geocode)
df['Lat'] = df['location'].apply(lambda x: x.latitude if x else None)
df['Lon'] = df['location'].apply(lambda x: x.longitude if x else None)

In [7]:
#Renaming the columns
df.rename(columns = {'add' : 'Nom_Barri'}, inplace = True)

In [8]:
#Creating the new dataframe
persones_new = persones.merge(df)
persones_new.rename(columns = {'Ocupacio_mitjana_(persones_ per_domicili)' : 'Ocupacio'}, inplace = True)

#A dataframe with the population per barri, which will be useful later
df1 = persones_new.groupby(['Nom_Barri'])['Poblacio'].sum()

Now we have a dataframe with the population of different districts and can locate them on our map.

In [9]:
#Viewing the new dataframe
persones_new

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Poblacio,Domicilis,Ocupacio,location,Lat,Lon
0,2021,1,Ciutat Vella,1,el Raval,48688,17298,2.81,"(el Raval, Ciutat Vella, Barcelona, Barcelonès...",41.379518,2.168368
1,2021,1,Ciutat Vella,2,el Barri Gòtic,22850,6571,3.48,"(el Gòtic, Ciutat Vella, Barcelona, Barcelonès...",41.383395,2.176912
2,2021,1,Ciutat Vella,3,la Barceloneta,15125,6927,2.18,"(la Barceloneta, Ciutat Vella, Barcelona, Barc...",41.380653,2.189927
3,2021,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",23009,10071,2.28,"(Sant Pere, Sant Pere, Santa Caterina i la Rib...",41.388322,2.177411
4,2021,2,Eixample,5,el Fort Pienc,33445,12782,2.62,"(el Fort Pienc, Eixample, Barcelona, Barcelonè...",41.397448,2.181713
...,...,...,...,...,...,...,...,...,...,...,...
68,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,13536,5356,2.53,"(Diagonal Mar, Diagonal Mar i el Front Marítim...",41.405228,2.213352
69,2021,10,Sant Martí,70,el Besòs i el Maresme,26116,8432,3.10,"(el Besòs i el Maresme, Sant Martí, Barcelona,...",41.414979,2.213877
70,2021,10,Sant Martí,71,Provençals del Poblenou,21185,8315,2.55,"(Provençals del Poblenou, La Escocesa, Sant Ma...",41.411948,2.204125
71,2021,10,Sant Martí,72,Sant Martí de Provençals,26229,10584,2.48,"(Sant Martí de Provençals, Sant Martí, Barcelo...",41.416519,2.198968


In [10]:
#We add the link which contain the GeoJson file to visualize our new data
BCNGeo = 'https://raw.githubusercontent.com/martgnz/bcn-geodata/master/barris/barris.geojson'

#Creating the new map
Barcelona_b = folium.Map(location=[41.39, 2.17], zoom_start=12,tiles='cartodbpositron')

Barcelona_b.choropleth(geo_data=BCNGeo, # GeoJson Coordinates
                   data=df1,   # The table which contains the values we are analysing
                   columns=["Nom_Barri","Poblacio"],
                   key_on='feature.properties.NOM', # We chose the key we need in the GeoJson file
                   fill_color='YlOrRd',
                   fill_opacity=0.7,
                   line_opacity=0.4,
                   legend_name='The number of inhabitants living in the Barri of Barcelona')


Barcelona_b

/Users/tgcajal/opt/anaconda3/lib/python3.8/site-packages/folium/folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


### Part 2: The Competition
Clearly, knowing our market size is not enough, we need to know how the competition is doing. Where are they located and how are they doing? According to New Economic Geography, the Fashion industry is a diffused one that tends to concentrate over time. Agglomeration advantages cause clothing stores to open close to one another. Ideally, our new shop will follow this logic. Let's see the competition on a map.

In [11]:
#Reading and viewing the data
shops = pd.read_csv('Shops.csv', sep = ';')
shops

,N,Name,Address,Latitude,Longitude
0,1,UNIQLO,NaN,41.389907,2.168345
1,2,Primark,"Сentro comercial Diagonal Mar, Avinguda Diagon...",41.410660,2.218103
2,3,Zara,NaN,41.385335,2.173205
3,4,COS,"Passeig de Gràcia 27, 08007 Barcelona Cataloni...",41.390907,2.166413
4,5,H&M,"Avinguda Portal de l'Àngel, 9, 08002 Barcelona...",41.385235,2.173605
5,6,C&A,NaN,41.385415,2.169018
6,7,Ropa de Hombre,NaN,41.407230,2.161494
7,8,Desigual Worldwide Headquarters,"Pg. Mare Nostrum, 15, 08039 Barcelona Cataloni...",41.369230,2.188816
8,9,Mango Outlet,"C. Girona, 37 (Gran Vía Corts Catalanes), 0801...",41.392456,2.173607
9,10,El Lokal,NaN,41.379394,2.167084


In [12]:
#Adding all shops to our initial map
Latitude = list(shops.Latitude)
Longitude = list(shops.Longitude)
for lat, lan in zip(Latitude,Longitude):
    folium.Marker([lat,lan]).add_to(Barcelona)
Barcelona

### Part 3: Where to?
Now that we can see our competition, we need to find available rentals for us to locate. However, this is not so easy, we will have to scrape the web for commercial rentals. Our client has chosen the search engine Habitaclia for this task. Let's see what we get.

In [13]:
# A function to get the data from Habitaclia with BeautifulSoup

def get_properties(link):
    """Web scraping the Habitaclia webpage to get the rental name, location, and price."""
    url = link
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    response = urlopen(req)    
    html = BeautifulSoup(response)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    rentals = html.find_all('h3', {'class':'list-item-title'})
    locations = html.find_all('p', {'class':'list-item-location'})
    prices = html.find_all('article', {'class':'list-item-price'})

    r = []

    for i in rentals:
        r.append(i.getText())

    l = []

    for i in locations:
        l.append(i.getText())

    p = []

    for i in prices:
        p.append(i.getText())
    
    zipped = list(zip(r,l,p))
    df = pd.DataFrame(zipped, columns=["Property","Location","Price"])

    return df

In [14]:
#A basic pagination method

pages = ["https://www.habitaclia.com/alquiler-locales_comerciales-barcelona.htm"]
x = 1

for i in range(10):
    pages.append("https://www.habitaclia.com/alquiler-locales_comerciales-barcelona-"+str(x)+".htm")
    x = x+1

In [15]:
#Pagination alternative, less automated

df_0 = get_properties(pages[0])
df_1 = get_properties(pages[1])
df_2 = get_properties(pages[2])
df_3 = get_properties(pages[3])
df_4 = get_properties(pages[4])
df_5 = get_properties(pages[5])
df_6 = get_properties(pages[6])
df_7 = get_properties(pages[7])
df_8 = get_properties(pages[8])
df_9 = get_properties(pages[9])
df_10 = get_properties(pages[10])

In [16]:
#Joining all properties from each paginated page
frames = [df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10]

available_properties = pd.concat(frames)
available_properties

,Property,Location,Price
0,\nAlquiler Local Comercial en Esquerra Baixa...,\nBarcelona - Esquerra Baixa de l´Eixample\n,\n3.000 €\n
1,\nAlquiler Local Comercial en Dreta de l´Eix...,\nBarcelona - Dreta de l´Eixample\n,\n7.000 €\n
2,\nAlquiler Local Comercial en Dreta de l´Eix...,\nBarcelona - Dreta de l´Eixample\n,\n2.400 €\n
3,\nAlquiler Local Comercial en Dreta de l´Eix...,\nBarcelona - Dreta de l´Eixample\n,\n5.350 €\nha bajado 250 €\n
4,\nAlquiler Local Comercial en St. Pere - Sta...,\nBarcelona - St. Pere - Sta. Caterina - El Bo...,\n12.000 €\n
...,...,...,...
10,\nAlquiler Local Comercial en Sagrada Famíli...,\nBarcelona - Sagrada Família\n,\n1.800 €\n
11,\nAlquiler Local Comercial en Carrer rossell...,\nBarcelona - Esquerra Alta de l´Eixample\n\n\...,\n2.150 €\nha bajado 250 €\n
12,\nAlquiler Local Comercial Arago\n,\nBarcelona - Sagrada Família\n,\n1.860 €\n
13,\nAlquiler Local Comercial en Tamarit 116. F...,\nBarcelona - Sant Antoni\n\n\n\n\nVer mapa\n\n,\n1.150 €\n


We now have a few pages of properties with their monthly prices and locations, but the data is not so clean. In order to clean it, we are going to extract the numbers from the Prices column, and reduce the amount of unnecessary characters from the Property column.

In [17]:
# Changing the thousand separator for our conversion to read them as such

strings = available_properties["Price"]
new_strings = []

for string in strings:
    new_string = string.replace(".","")
    new_strings.append(new_string)

In [18]:
# Adding the new price column

prices_num = []

for i in new_strings:
    for x in i.split():
        if x.isdigit():
            num = int(x)

    prices_num.append(num)

available_properties["Price EUR"] = prices_num

In [19]:
# Eliminating unnecessary words and characters

col_0 = available_properties.iloc[:,0]
col_0 = list(map(lambda x: x.replace('Alquiler','').replace('Local','').replace('Comercial','').replace('comercial','').replace('\n','').replace('en',''), col_0))

available_properties["Property"] = col_0

Not all properties include a useful adress in their announcements, but our client does not have time to contact each announcer and ask for specifics, so we are adding coordinates where possible and dropping the rest to filter.

In [20]:
#New dataframe with coordinates
df = pd.DataFrame({'add': available_properties["Property"]}) 
geolocator = Nominatim(user_agent="my_request")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df['location'] = df['add'].apply(geocode)
df['Lat'] = df['location'].apply(lambda x: x.latitude if x else None)
df['Lon'] = df['location'].apply(lambda x: x.longitude if x else None)
df

RateLimiter caught an error, retrying (0/2 tries). Called with (*('      Carrer tamarit, 96.    zona de paso',), **{}).
Traceback (most recent call last):
  File "/Users/tgcajal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/tgcajal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/tgcajal/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/tgcajal/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/tgcajal/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/tgcajal/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
  

,add,location,Lat,Lon
0,Esquerra Baixa de l´Eixample. Luminoso y...,None,NaN,NaN
1,Dreta de l´Eixample. Excelte ubicacion,None,NaN,NaN
2,Dreta de l´Eixample. Muy bi comunicado-z...,None,NaN,NaN
3,Dreta de l´Eixample. En perfecto estado,None,NaN,NaN
4,St. Pere - Sta. Caterina - El Born. Espa...,None,NaN,NaN
...,...,...,...,...
10,Sagrada Família. vta y alquiler barc...,None,NaN,NaN
11,"Carrer rossello, 25. Diáfano y con gran ...",None,NaN,NaN
12,Arago,"(Aragón, España, (41.3787291, -0.7639373))",41.378729,-0.763937
13,Tamarit 116. Fabuloso local de 139m2 sa...,None,NaN,NaN


In [21]:
#Clean dataframe of the properties to be considered with the essential columns
final_options = pd.concat([available_properties, df], axis=1)
final_options = final_options.drop(columns=["Location","Price","add","location"])
final_options = final_options.dropna()
final_options

,Property,Price EUR,Lat,Lon
5,Balmes 185,5800,-33.042403,-71.633455
8,Tres Torres,3500,41.599268,2.283601
14,Guitard 15,200,43.584459,1.488279
0,Dreta de l´Eixample,10000,41.394128,2.166465
2,Sant Gervasi - Bonanova,9500,41.403329,2.137779
3,"Carrer lledo, 17. el gótico",1300,41.382162,2.179312
5,La rambla,61041,41.385348,2.170387
10,Carrer del camp,2300,41.401186,2.138671
9,Carrer de bilbao.,1200,39.568659,-0.518130
12,Rambla guipuscoa (de),1700,41.415219,2.195594


### Part 4: Tying it all together
It is finally time to tie everything together. First, we are calculating which of our final options are closer to the hot spots of the fashion scene in Barcelona. Then, we are locating our top 3 options on the population heat map, we are making sure that our choice is located within the most populated barri of Barcelona. Finally, we will pick a property based on the price, since the goal is to minimize costs while maximizing the revenue, of course.

In [22]:
#Getting the market size by neighborhood
mkt_size = persones_new.groupby("Nom_Districte")["Poblacio"].sum()
mkt_size

Nom_Districte
Ciutat Vella           109672
Eixample               269349
Gràcia                 123276
Horta-Guinardó         173944
Les Corts               81576
Nou Barris             173552
Sant Andreu            151537
Sant Martí             241181
Sants-Montjuïc         187026
Sarrià-Sant Gervasi    149201
Name: Poblacio, dtype: int64

In [23]:
#Applying a very simple euclidean distance calculation (continued in the next cell)
mean_lat = shops["Latitude"].mean()
mean_lon = shops["Longitude"].mean()

In [24]:
final_options["lat_diff"] = (mean_lat - final_options["Lat"])/final_options["Lat"]*100
final_options["lon_diff"] = (mean_lon - final_options["Lon"])/final_options["Lon"]*100

#Viewing the final dataframe with distances to retail clusters
final_options

,Property,Price EUR,Lat,Lon,lat_diff,lon_diff
5,Balmes 185,5800,-33.042403,-71.633455,-225.272761,-103.031597
8,Tres Torres,3500,41.599268,2.283601,-0.495529,-4.902946
14,Guitard 15,200,43.584459,1.488279,-5.027773,45.915964
0,Dreta de l´Eixample,10000,41.394128,2.166465,-0.002409,0.238758
2,Sant Gervasi - Bonanova,9500,41.403329,2.137779,-0.024631,1.583806
3,"Carrer lledo, 17. el gótico",1300,41.382162,2.179312,0.026506,-0.352158
5,La rambla,61041,41.385348,2.170387,0.018808,0.057620
10,Carrer del camp,2300,41.401186,2.138671,-0.019456,1.541423
9,Carrer de bilbao.,1200,39.568659,-0.518130,4.610902,-519.129909
12,Rambla guipuscoa (de),1700,41.415219,2.195594,-0.053333,-1.091118


In [25]:
#Filtering top three options by location and price
top_3 = final_options.sort_values(["lat_diff", "lon_diff", "Price EUR"], key=pd.Series.abs)[:3]
top_3

,Property,Price EUR,Lat,Lon,lat_diff,lon_diff
0,Dreta de l´Eixample,10000,41.394128,2.166465,-0.002409,0.238758
5,"Carrer alt de gironella, 3",400,41.396213,2.132172,-0.007444,1.850957
5,La rambla,61041,41.385348,2.170387,0.018808,0.057620


In [26]:
#Adding the top options to the map
Latitudes = list(top_3.Lat)
Longitudes = list(top_3.Lon)
for lat, lan in zip(Latitudes,Longitudes):
    folium.Marker([lat,lan]).add_to(Barcelona_b)

Barcelona_b

**As the map shows, our top property will be in Eixample, one of the top populated barri, very close to the cluster of shops in the city. Our client will be paying the lowest available price for a perfect location!**